In [1]:
import os 
os.chdir('../../')
os.environ["DPM_TQDM"] = "False"

In [2]:
from backbones.dit import DiT

# 사용 예
model = DiT(trainable=True)
print(model)

/home/scpark/miniconda3/envs/sana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/scpark/miniconda3/envs/sana/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/scpark/miniconda3/envs/sana/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
Loading pipeline components...:   0%

In [3]:
import torch
import torch.nn.functional as F
from torch import amp
from solvers.dual.static.gdual_box_solver import GDual_Box_Solver

# 클래스 0–9로 설정
class_ids = [i for i in range(8)]
model_fn, noise_schedule, latents = model.get_model_fn(pos_conds=class_ids, guidance_scale=3.0)

device = 'cuda:0'
solver = GDual_Box_Solver(model_fn, noise_schedule, steps=10, skip_type="time_uniform")
solver = solver.to(device)
print('done')

optimizer = torch.optim.Adam(solver.parameters(), lr=1e-3)
scaler = amp.GradScaler()
print('done')

done
done


In [12]:
optimizer.zero_grad()
with amp.autocast(device_type=device, dtype=torch.bfloat16):
    pr_latents = solver.sample(latents)
    gt_latents = torch.randn_like(pr_latents)
    loss = F.mse_loss(pr_latents, gt_latents)

scaler.scale(loss).backward()
scaler.step(optimizer)
scaler.update()

print('done')

done


In [13]:
solver.params

Parameter containing:
tensor([[ 0.0016,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0011,  0.0032,  0.0020,  0.0022, -0.0013],
        [-0.0041,  0.0026,  0.0028,  0.0028, -0.0019],
        [ 0.0018,  0.0045,  0.0010,  0.0049, -0.0049],
        [ 0.0050, -0.0049, -0.0050, -0.0049,  0.0050],
        [ 0.0036, -0.0046, -0.0042, -0.0046,  0.0042],
        [ 0.0049, -0.0050, -0.0050, -0.0050,  0.0050],
        [ 0.0048, -0.0049, -0.0049, -0.0049,  0.0049],
        [ 0.0050, -0.0050, -0.0050, -0.0050,  0.0050],
        [ 0.0004,  0.0000,  0.0000,  0.0000,  0.0000]], device='cuda:0',
       requires_grad=True)